# **NOTEBOOK 3: MODELADO**

#### En este notebook probaremos distintos modelos de Machine Learning para lograr nuestro objetivo: **predecir si un cliente abandonará o no la empresa de telecomunicaciones con la que estamos trabajando**.

In [37]:
# Cargamos de nuevo nuestro dataset limpio
import pandas as pd

df = pd.read_csv('df_clean.csv')

#### Para codificar nuestras variables categóricas, usaremos One-Hot Encoding para aquellas que tengan más de dos posibles valores y LabelEncoding para aquellas que tengan tan sólo 2 posibles valores.

In [38]:
# Definimos una función para contar los posibles valores de cada variable categórica
def contar_valores(df):
    for columna in df.columns:
        if df[columna].dtype == 'object':
            cuantos = df[columna].nunique()
            print(f"{columna}: {cuantos} valores diferentes")

contar_valores(df)

customerID: 7043 valores diferentes
gender: 2 valores diferentes
Partner: 2 valores diferentes
Dependents: 2 valores diferentes
PhoneService: 2 valores diferentes
MultipleLines: 3 valores diferentes
InternetService: 3 valores diferentes
OnlineSecurity: 3 valores diferentes
StreamingTV: 3 valores diferentes
Contract: 3 valores diferentes
PaperlessBilling: 2 valores diferentes
PaymentMethod: 4 valores diferentes
Churn: 2 valores diferentes


In [39]:
# Aplicamos Label Encoding para las variables binarias
from sklearn.preprocessing import LabelEncoder

var_bin = df[['gender', 'Partner', 'Dependents', 'PhoneService', 'PaperlessBilling']]

le = LabelEncoder()

for col in var_bin:
    df[col] = le.fit_transform(df[col])

In [40]:
# Aplicamos One-Hot Encoding para el resto de variables categóricas
df = pd.get_dummies(df,
                     columns = ['MultipleLines', 'InternetService', 'OnlineSecurity', 'StreamingTV', 'Contract', 'PaymentMethod'],
                     drop_first = True)

In [41]:
# Comprobamos que se creen las columnas
df.columns

Index(['customerID', 'gender', 'SeniorCitizen', 'Partner', 'Dependents',
       'tenure', 'PhoneService', 'PaperlessBilling', 'MonthlyCharges',
       'TotalCharges', 'Churn', 'MultipleLines_No phone service',
       'MultipleLines_Yes', 'InternetService_Fiber optic',
       'InternetService_No', 'OnlineSecurity_No internet service',
       'OnlineSecurity_Yes', 'StreamingTV_No internet service',
       'StreamingTV_Yes', 'Contract_One year', 'Contract_Two year',
       'PaymentMethod_Credit card (automatic)',
       'PaymentMethod_Electronic check', 'PaymentMethod_Mailed check'],
      dtype='object')

In [42]:
# Eliminamos del modelo los customerID (no aportan información)
customer_ids = df['customerID']

df = df.drop(columns='customerID')

In [43]:
# Convertimos los valores Booleans en 1 y 0
bool_cols = df.select_dtypes(include='bool').columns
df[bool_cols] = df[bool_cols].astype(int)

In [44]:
# Convertimos Churn a 1 y 0 (1 si es Churn, que es lo que queremos detectar)
df['Churn'] = df['Churn'].map({'Yes': 1, 'No': 0})

In [45]:
# Seleccionamos nuestra columnas numéricas
num_cols = ['tenure', 'MonthlyCharges', 'TotalCharges']

# Separamos la variable objetivo del resto de variables
X = df.drop(columns='Churn')
y = df['Churn']

In [46]:
# Las escalamos con StandardScaler para no tener valores muy dispersos
num_cols = ['tenure', 'MonthlyCharges', 'TotalCharges']

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X[num_cols] = scaler.fit_transform(X[num_cols])

In [47]:
# Comprobamos que solo tenemos variables numéricas
df.dtypes

gender                                     int64
SeniorCitizen                              int64
Partner                                    int64
Dependents                                 int64
tenure                                     int64
PhoneService                               int64
PaperlessBilling                           int64
MonthlyCharges                           float64
TotalCharges                             float64
Churn                                      int64
MultipleLines_No phone service             int64
MultipleLines_Yes                          int64
InternetService_Fiber optic                int64
InternetService_No                         int64
OnlineSecurity_No internet service         int64
OnlineSecurity_Yes                         int64
StreamingTV_No internet service            int64
StreamingTV_Yes                            int64
Contract_One year                          int64
Contract_Two year                          int64
PaymentMethod_Credit

#### En este momento, nuestro DataFrame está **preparado** para empezar a ser entrenado y testeado por diferentes modelos y algoritmos.

#### Para realizar la división del DataFrame, dividiremos en test y entrenamiento, así como en validación y entrenamiento.

In [48]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size = 0.2,
    random_state = 42,
    stratify = y
)

In [49]:
X_train, X_val, y_train, y_val = train_test_split(
    X_train,
    y_train,
    test_size=0.25,
    random_state=42,
    stratify=y_train
)

#### **Prueba de modelos**

Empezaremos por los 3 siguientes:
- Regresión Logística: *la variable objetivo es categórica, por lo que puede ser adecuada. Además, probar con un modelo sencillo siempre es necesario para ver si realmente tenemos que escalar a algoritmos más complejos.*

- Decission Tree Clasifier: *a diferencia de la regresión logística, puede capturar relaciones no lineales, pero debemos tener cuidado con el overfitting.*

- Random Forest: *más complejo y lento, pero puede reducir el overfitting.*

In [50]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, classification_report

## Regresión Logística

In [52]:
reg_log = LogisticRegression(random_state = 42)
reg_log.fit(X_train, y_train)

# Probamos con el conjunto de validación
y_pred_val = reg_log.predict(X_val)

In [53]:
print("Accuracy:", accuracy_score(y_val, y_pred_val))
print("Precision:", precision_score(y_val, y_pred_val))
print("Recall:", recall_score(y_val, y_pred_val))
print("F1-Score:", f1_score(y_val, y_pred_val))
print("\nConfusion Matrix:\n", confusion_matrix(y_val, y_pred_val))

Accuracy: 0.8062455642299503
Precision: 0.6711864406779661
Recall: 0.5294117647058824
F1-Score: 0.5919282511210763

Confusion Matrix:
 [[938  97]
 [176 198]]


## Decission Tree Classiffier

In [56]:
dec_tree = DecisionTreeClassifier(
    max_depth=5,           # Profundidad máxima del árbol
    min_samples_split=20,  # Mínimo de muestras para dividir un nodo
    min_samples_leaf=10,   # Mínimo de muestras en una hoja
    random_state = 42)
dec_tree.fit(X_train, y_train)

# Probamos con el conjunto de validación
y_pred_val = dec_tree.predict(X_val)

In [57]:
print("Accuracy:", accuracy_score(y_val, y_pred_val))
print("Precision:", precision_score(y_val, y_pred_val))
print("Recall:", recall_score(y_val, y_pred_val))
print("F1-Score:", f1_score(y_val, y_pred_val))
print("\nConfusion Matrix:\n", confusion_matrix(y_val, y_pred_val))

Accuracy: 0.7764371894960965
Precision: 0.6104868913857678
Recall: 0.4358288770053476
F1-Score: 0.5085803432137286

Confusion Matrix:
 [[931 104]
 [211 163]]


## Random Forest

In [58]:
rd_for = RandomForestClassifier(random_state = 42)

rd_for.fit(X_train, y_train)

y_pred_val = rd_for.predict(X_val)

In [59]:
print("Accuracy:", accuracy_score(y_val, y_pred_val))
print("Precision:", precision_score(y_val, y_pred_val))
print("Recall:", recall_score(y_val, y_pred_val))
print("F1-Score:", f1_score(y_val, y_pred_val))
print("\nConfusion Matrix:\n", confusion_matrix(y_val, y_pred_val))

Accuracy: 0.7899219304471257
Precision: 0.6466165413533834
Recall: 0.45989304812834225
F1-Score: 0.5375

Confusion Matrix:
 [[941  94]
 [202 172]]
